**INSTALLING THE RASTERIO MODULE**

In [ ]:
!pip install rasterio

**LOADING THE NECESSARY PACKAGES**

**geopandas (imported as gpd)**: A Python library used for working with geospatial data. It extends the functionalities of pandas, allowing us to work with geospatial data in GeoDataFrame structures, which are like pandas DataFrames but with additional geospatial capabilities. In this script, it's used to read, manipulate, and analyze geospatial data from shapefiles.

**ee (Earth Engine):** This is the Google Earth Engine library, which provides access to a massive catalog of satellite imagery and geospatial datasets. It's used for advanced geospatial data processing and analysis tasks, particularly working with satellite data and performing large-scale, cloud-based analysis.

**pandas (imported as pd):** A foundational library for data manipulation and analysis in Python, offering powerful data structures like DataFrames. It's widely used for data cleaning, transformation, and analysis tasks. In this script, it might be used for handling non-geospatial data processing tasks or in conjunction with geopandas for certain operations.

**os: **This is a standard Python library that provides a way of using operating system dependent functionality like reading, writing, and manipulating paths and files. In geospatial scripts, it's often used for file path manipulations and directory operations.

**rasterio:** A library for reading and writing raster datasets (like GeoTIFFs). It's often used in geospatial data processing for working with raster data, which includes satellite imagery and other grid-based data formats.

**shapely:** A library for manipulation and analysis of planar geometric objects. It is primarily used in Python for spatial analysis and geometric operations, like calculating areas, distances, intersections, and unions of geospatial objects.

**geemap:** A Python package for interactive mapping with Google Earth Engine (GEE), which integrates GEE with IPython and Jupyter notebook. It's used to facilitate mapping and visualization of geospatial data processed with GEE, and can also provide additional functionalities for exporting data from GEE.

In [2]:
import geopandas as gpd
import ee
import pandas as pd
import os
import rasterio
import shapely
import geemap

**AUTHENTICATING EARTH ENGINE**

In [ ]:
ee.Authenticate()


**INITIALIZING EARTH ENGINE**

In [4]:
ee.Initialize()

**Function Name: earth_engine_calculate_normals**

**Purpose:** This function calculates the monthly average (normals) of a specified band from a satellite dataset within a given region of interest (ROI) over a specified time period. It exports these averages as GeoTiff files.

**Parameters:**

**dataset:** The name or ID of the satellite image collection from Google Earth Engine (e.g., a specific satellite dataset).

**start_date:** The start date for the data collection period (string format expected, e.g., 'YYYY-MM-DD').

**end_date:** The end date for the data collection period (string format).

**band_name:** The name of the specific band in the satellite dataset to be analyzed.

**roi (Region of Interest):** A ee.Geometry object defining the geographical area for which the data is to be processed.

**output_dir:** The directory path where the exported GeoTiff files will be saved.

**scale:** The scale in meters for the exported images, which determines the pixel resolution.

**Process:**

1) The function loads the specified image collection from Earth Engine, filtering it by the given dates, band, and geographical bounds.

2) It calculates the average value of the specified band for each month across all years in the given date range.

3) These monthly averages are then exported as GeoTiff files, one for each month, to the specified output directory.

**Output:**

The function returns a list of file paths, each corresponding to an exported GeoTiff file containing the monthly average for the specified band. The files are named in the format Normal_{month}.tif, where {month} is the month number (1 to 12).

**Usage Notes:**

Ensure that the Google Earth Engine Python API is authenticated and initialized in your environment before using this function.

The output_dir should have write permissions, and the function will append a '/' if not already present at the end of the path to correctly format the file paths.

The function prints a message to the console each time a file is successfully exported.

In [7]:
def earth_engine_calculate_normals(dataset, start_date, end_date, band_name, roi, output_dir, scale):
    # Load in the dataset and filter it based on given parameters
    dataset = ee.ImageCollection(dataset).filterDate(start_date, end_date).select(band_name).filterBounds(roi)

    # Preparing a dictionary to hold the monthly normals
    monthly_normals = {}

    # Loop through each month
    for month in range(1, 13):
        # Filter the dataset for the specific month across all years
        month_filter = ee.Filter.calendarRange(month, month, 'month')
        monthly_data = dataset.filter(month_filter)

        # Compute the average for this month across all years
        monthly_avg = monthly_data.mean()

        # Store the result in the dictionary
        monthly_normals[month] = monthly_avg

    # Initialize the list to store exported file paths
    monthly_normals_export_list = []

    # Ensure the output directory ends with a slash
    if not output_dir.endswith('/'):
        output_dir += '/'

    for month, image in monthly_normals.items():
        # Construct the output path
        out_path = f'{output_dir}Normal_{month}.tif'

        # Export the image using geemap's export function
        geemap.ee_export_image(image, filename=out_path, scale=scale, region=roi, file_per_band=False)

        # Append the output path to the list
        monthly_normals_export_list.append(out_path)
        print(f'Exported Normal_{month}.tif')

    return monthly_normals_export_list





**Function Name: main**

**Purpose:** This function serves as the primary entry point for a script that processes and analyzes geospatial data. It reads a shapefile to determine a region of interest (ROI), extracts the bounding box of this region, and then uses this ROI to calculate monthly normals for a specific dataset and band within the specified time period. The results are exported as GeoTiff files.

**Process:**

**Read Shapefile: **The function begins by reading a shapefile (containing geographical data) using GeoPandas.

**Coordinate Reference System Transformation:** It then transforms the GeoDataFrame to the EPSG:4326 coordinate reference system for standardized latitude and longitude coordinates.
Filter GeoDataFrame: The function filters the GeoDataFrame to include only the features with a specific attribute ('USSOUTHCOM' in the 'cocom' column).

**Combine Geometries:** All geometries in the filtered GeoDataFrame are combined into a single unified geometry.

**Calculate Bounding Box:** The bounding box (geographical bounds) of this combined geometry is calculated.

**Create Earth Engine Geometry: **An Earth Engine Rectangle geometry is created using the bounding box coordinates. This defines the ROI for the Earth Engine data processing.

**Set Parameters for Data Processing:** The function defines various parameters including the dataset, start and end dates, band name, ROI, output directory, and scale.

**Call earth_engine_calculate_normals Function:** These parameters are passed to the earth_engine_calculate_normals function to calculate the monthly normals and export them as GeoTiff files.

**Output Information: **After processing, the function prints the paths of the exported files.

**Parameters:**

The function does not take any parameters but relies on hard-coded values within its scope. These include file paths, dataset identifiers, date ranges, and other specific settings related to the geospatial analysis.

**Outputs:**

Prints the file paths of the exported GeoTiff files to the console.

**Usage Notes:**

Ensure all necessary libraries (geopandas, ee, etc.) are installed and properly configured.
The shapefile path and other parameters like dataset name, band name, and output directory should be modified according to the user's specific requirements.

The script should be executed in an environment where the Google Earth Engine API is authenticated.

In [9]:
def main():
    # Read the shapefile
    gdf = gpd.read_file('/content/drive/MyDrive/COM_shapefiles/cocoms.shp')

    # Transform the GeoDataFrame to EPSG:4326
    gdf = gdf.to_crs(epsg=4326)

    # Load and filter your GeoDataFrame
    filtered_gdf = gdf[gdf['cocom'] == 'USSOUTHCOM']

    # Combine all geometries into a single geometry
    combined_geom = filtered_gdf.geometry.unary_union

    # Get the bounding box of the combined geometry
    minx, miny, maxx, maxy = combined_geom.bounds

    # Create an Earth Engine Geometry for the bounding box
    roi = ee.Geometry.Rectangle([minx, miny, maxx, maxy])

    # Define other parameters for the earth_engine_calculate_normals function
    dataset = 'ECMWF/ERA5/MONTHLY'
    start_date = '1990-01-01' ### Define start date
    end_date = '2019-12-01' ## Define the end date
    band_name = 'mean_2m_air_temperature' # Define the band of interest
    roi = roi ## region of interest
    output_dir = '/content/drive/MyDrive/ERA_monthly_normals/'
    scale = 27830 ## this is in metres

    # Call the earth_engine_calculate_normals function
    exported_files = earth_engine_calculate_normals(dataset, start_date, end_date, band_name, roi, output_dir, scale)

    # Optionally, do something with the returned list of exported files
    print("Exported files:", exported_files)

# Call the main function
if __name__ == "__main__":
    main()

Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/ERA_monthly_normals/Normal_1.tif
Exported Normal_1.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/ERA_monthly_normals/Normal_2.tif
Exported Normal_2.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/ERA_monthly_normals/Normal_3.tif
Exported Normal_3.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/ERA_monthly_normals/Normal_4.tif
Exported Normal_4.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/ERA_monthly_normals/Normal_5.tif
Exported Normal_5.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/ERA_monthly_normals/Normal_6.tif
Exported Normal_6.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/ERA_monthly_normals/Normal_7.tif
Exported Normal_7.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive